In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import numpy as np
import glob
from ast import literal_eval
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
# Find all chunk files (assuming they follow a consistent naming pattern)
file_paths = glob.glob('../datasets/2025_csv/_chunk_*.csv')
# Read each file (limiting rows if necessary) and concatenate
dfs = [pd.read_csv(fp, nrows=10000) for fp in file_paths]
df = pd.concat(dfs, ignore_index=True)

# Printing the original columns
print("Dataset columns:")
print(df.columns)
print(df.head)

Dataset columns:
Index(['$insert_id', 'amplitude_id', 'app', 'city', 'client_event_time',
       'client_upload_time', 'country', 'data', 'data_type', 'device_family',
       'device_id', 'device_type', 'dma', 'event_id', 'event_properties',
       'event_time', 'event_type', 'language', 'library', 'os_name',
       'os_version', 'platform', 'processed_time', 'region',
       'server_received_time', 'server_upload_time', 'session_id', 'user_id',
       'user_properties', 'uuid'],
      dtype='object')
<bound method NDFrame.head of                                   $insert_id   amplitude_id     app  \
0       74413f76-5071-4c44-bd3b-5ded989dec39  1090135225326  591532   
1       9c88e80f-9292-459b-b1df-1be14f5c8235  1090135225326  591532   
2       5679fe11-d8cd-4951-94be-9ee3e1c7179b  1090135225326  591532   
3       00c40271-0d81-4fa9-aad7-9f06929df50f  1090135225326  591532   
4       0120d0cc-94fd-4050-b61b-2f610fce9bcf  1090135225326  591532   
...                                  

In [4]:
# Code checks which columns may have nested JSON data
print("Columns with potential JSON data:")
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            sample = df[col].dropna().iloc[0]
            if sample.startswith('{') or sample.startswith('['):
                print(f"- {col} (Possible JSON)")
        except Exception:
            pass

Columns with potential JSON data:
- data (Possible JSON)
- event_properties (Possible JSON)
- user_properties (Possible JSON)


In [5]:
# Ensure each value is a dictionary
# Determine the union of all keys
# Create new columns for each key
# Drop the original nested JSON columns
def ensure_dict(x):
    if isinstance(x, dict):
        return x
    try:
        # String representation of a dict turned into dictionary object
        return literal_eval(x)
    except Exception:
        # If parsing fails, return an empty dict
        return {}

In [6]:
# List of nested columns to process
nested_columns = ['data', 'event_properties', 'user_properties']

# Process each nested column
# ONLY RUN THIS ONCE!
for col in nested_columns:
    # Ensure each value in the column is a dictionary
    df[col] = df[col].apply(ensure_dict)
    
    # Determine the union of all keys in the column
    all_keys = set()
    df[col].dropna().apply(lambda d: all_keys.update(d.keys()) if isinstance(d, dict) else None)
    
    # For each key, create a new column only if it has at least one valid (non-empty) value
    for key in all_keys:
        new_col_name = f"{col}_{key}"
        # Create a temporary Series for this key
        series = df[col].apply(lambda d: d.get(key, None) if isinstance(d, dict) else None)
        # Check if there's at least one valid value (not None, {}, [], or 'EMPTY')
        if series.apply(lambda v: v not in (None, {}, [], 'EMPTY')).any():
            df[new_col_name] = series
    
    # After processing, drop the original nested JSON column.
    df.drop(columns=[col], inplace=True)

# Convert specified timestamp columns to datetime objects
time_cols = ['client_event_time', 'event_time', 'server_received_time']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [7]:
# Printing the columns (new additional ones added from flattening nested columns)
print("Dataset columns:")
print(df.columns)
print("# of columns:", len(df.columns))

Dataset columns:
Index(['$insert_id', 'amplitude_id', 'app', 'city', 'client_event_time',
       'client_upload_time', 'country', 'data_type', 'device_family',
       'device_id', 'device_type', 'dma', 'event_id', 'event_time',
       'event_type', 'language', 'library', 'os_name', 'os_version',
       'platform', 'processed_time', 'region', 'server_received_time',
       'server_upload_time', 'session_id', 'user_id', 'uuid', 'data_path',
       'data_user_properties_updated', 'event_properties_policy-id',
       'event_properties_[Amplitude] Session Replay ID',
       'event_properties_emailId', 'event_properties_templateName',
       'event_properties_menu', 'event_properties_displayName',
       'event_properties_messageId', 'event_properties_hasAccounts',
       'event_properties_attachmentId', 'event_properties_hasAssignees',
       'event_properties_id', 'event_properties_type',
       'event_properties_line-of-business', 'event_properties_account-id',
       'event_properties_qu

In [8]:
# Single example looks like:
# Display entire column content without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df.iloc[140].to_string())

$insert_id                                                     45cc9221-ce22-48ac-9906-c44948221d00
amplitude_id                                                                          1090135225326
app                                                                                          591532
city                                                                                            NaN
client_event_time                                                        2025-01-13 20:53:30.155000
client_upload_time                                                       2025-01-13 20:53:31.291000
country                                                                               United States
data_type                                                                                     event
device_family                                                                               Windows
device_id                                                      76c54070-44c1-461f-854c-8c3082635877


In [9]:
# What does atleast a single valid entry look like?
# Dictionary to store a valid value for each column
valid_entries = {}
# Iterate over each column in the DataFrame
for col in df.columns:
    # Iterate over each row value for this column using .items()
    for idx, value in df[col].items():
        # Check if the value is not empty (None, empty dict, empty list, or 'EMPTY')
        if value not in (None, {}, [], 'EMPTY'):
            # Once a valid value is found, record it and its type, then break out of the loop for this column
            valid_entries[col] = (value, type(value))
            break

# Print the results
for col, (value, dtype) in valid_entries.items():
    print(f"Column: {col}\n  Value: {value}\n  Type: {dtype}\n")
print(len(valid_entries))

Column: $insert_id
  Value: 74413f76-5071-4c44-bd3b-5ded989dec39
  Type: <class 'str'>

Column: amplitude_id
  Value: 1090135225326
  Type: <class 'int'>

Column: app
  Value: 591532
  Type: <class 'int'>

Column: city
  Value: nan
  Type: <class 'float'>

Column: client_event_time
  Value: 2025-01-13 20:32:46.571000
  Type: <class 'pandas._libs.tslibs.timestamps.Timestamp'>

Column: client_upload_time
  Value: 2025-01-13 20:32:47.664000
  Type: <class 'str'>

Column: country
  Value: United States
  Type: <class 'str'>

Column: data_type
  Value: event
  Type: <class 'str'>

Column: device_family
  Value: Windows
  Type: <class 'str'>

Column: device_id
  Value: 76c54070-44c1-461f-854c-8c3082635877
  Type: <class 'str'>

Column: device_type
  Value: Windows
  Type: <class 'str'>

Column: dma
  Value: nan
  Type: <class 'float'>

Column: event_id
  Value: 6023
  Type: <class 'int'>

Column: event_time
  Value: 2025-01-13 20:32:46.571000
  Type: <class 'pandas._libs.tslibs.timestamps.Ti

In [10]:
# Check that we dont have columns with zero valid pieces of data
# First, determine the columns with valid data (as already stored in valid_entries)
valid_columns = set(valid_entries.keys())
# Get all DataFrame columns
all_columns = set(df.columns)
# Determine columns that did not yield any valid entry
invalid_columns = all_columns - valid_columns
print("Total number of columns in df:", len(df.columns))
print("Number of columns with at least one valid entry:", len(valid_columns))
print("Number of columns with NO valid data:", len(invalid_columns))
print("Columns with no valid data:")
for col in invalid_columns:
    print("  -", col)


Total number of columns in df: 67
Number of columns with at least one valid entry: 67
Number of columns with NO valid data: 0
Columns with no valid data:


### Prediction Modelling

In [11]:
# Convert event_time to datetime and sort
df['event_time'] = pd.to_datetime(df['event_time'])
df.sort_values(by=['session_id', 'event_time'], inplace=True)  # Changed to session_id

In [12]:
# Group by session_id instead of user_id
session_groups = df.groupby('session_id')

In [13]:
features = [
    "city", "client_event_time", "client_upload_time", "country", "data_type",
    "device_family", "device_type", "language", "event_type", "os_name", "os_version",
    "platform", "region", "event_properties_type", "event_properties_line-of-business",
    "event_properties_accountId", "event_properties_status", "event_properties_lineOfBusiness",
    "event_properties_action", "event_properties_error", "event_properties_templateName",
    "event_properties_hasAssignees", "event_properties_hasAccounts", "event_properties_displayName",
    "event_properties_variant", "event_properties_tableId", "event_properties_menu",
    "event_properties_filename", "event_properties_attachmentId", "user_properties_isInternalUser",
    "user_properties_roles", "user_properties_referring_domain", "user_properties_businessUnit",
    "user_properties_hostname"
]
print(len(features))

34


In [14]:
# Create index mappings for each feature (modified for list handling)
feature_to_idx = {}

for feat in features:
    if feat == 'user_properties_roles':
        # Handle list values by taking first role
        unique_vals = (
            df[feat]
            .dropna()
            .apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else np.nan)
            .dropna()
            .astype(str)
            .unique()
            .tolist()
        )
    else:
        unique_vals = df[feat].dropna().astype(str).unique().tolist()
    
    # Create mapping with default index 0
    idx_map = {'__default__': 0}
    current_idx = 1
    for val in unique_vals:
        key = str(val)
        if key not in idx_map:
            idx_map[key] = current_idx
            current_idx += 1
    feature_to_idx[feat] = idx_map

In [15]:
feature_to_idx['user_properties_roles']

{'__default__': 0,
 'underwriter': 1,
 'admin': 2,
 'agent': 3,
 'System Admin': 4,
 'Broker': 5,
 'manager': 6,
 'Broker View-only': 7,
 'Underwriter': 8,
 'Leader': 9,
 'System Manager': 10,
 'executive': 11,
 'FS': 12}

In [16]:
# Process data into indices (with error handling)
concatenated_sequences = {}

for session_id, group in session_groups:  # Now using session_id
    group_sorted = group.sort_values(by='event_time')
    event_indices = []
    
    for _, row in group_sorted.iterrows():
        indices = []
        for feat in features:
            val = row[feat]
            
            # Handle null/empty values
            if isinstance(val, (list, np.ndarray)):
                
                # Handle empty lists/arrays
                if len(val) == 0:
                    indices.append(0)
                    continue
                
                # Special handling for user_properties_roles
                if feat == 'user_properties_roles':
                    # Take first role if available
                    val = val[0] if len(val) > 0 else np.nan
                else:
                    # Convert list to string representation
                    val = str(val)
            
            elif pd.isnull(val):
                indices.append(0)
                continue
            else:
                val = str(val)
            
            # Get index from mapping
            idx = feature_to_idx[feat].get(val, 0)
            indices.append(idx)
        
        event_indices.append(torch.tensor(indices, dtype=torch.long))
    
    if event_indices:
        concatenated_sequences[session_id] = event_indices
print(len(concatenated_sequences))

8931


In [17]:
# Set the window size and maximum examples
time_steps = 8
max_examples = 133713371337

# Create action to index mapping
action_feat = "event_type"
action_vals = df[action_feat].dropna().astype(str).unique().tolist()
action_to_idx = {val: idx for idx, val in enumerate(action_vals)}

# Initialize lists to store the examples and labels
X_examples = []
y_examples = []

too_short_count = 0
invalid_label_count = 0
total_sessions = len(concatenated_sequences)

for session_id, events in concatenated_sequences.items():
    if len(events) <= time_steps:
        too_short_count += 1
        continue  # Skip too short sessions
    
    session_df = df[df['session_id'] == session_id].sort_values(by='event_time')
    
    for i in range(len(events) - time_steps):
        window = events[i:i + time_steps]
        example = torch.stack(window)  # shape: (time_steps, hidden_size)
        
        target_row = session_df.iloc[i + time_steps]
        if pd.isnull(target_row[action_feat]):
            invalid_label_count += 1
            continue  # Skip missing labels
        
        target_val = str(target_row[action_feat])
        if target_val not in action_to_idx:
            invalid_label_count += 1
            continue  # Skip unknown labels
        
        label_idx = action_to_idx[target_val]
        
        X_examples.append(example)
        y_examples.append(label_idx)
        
        if len(X_examples) >= max_examples:
            break
    if len(X_examples) >= max_examples:
        break

# Print debugging info
print(f"Total sessions: {total_sessions}")
print(f"Too short sessions (≤{time_steps}): {too_short_count}")
print(f"Invalid or missing labels: {invalid_label_count}")
print(f"Valid examples collected: {len(X_examples)}")

Total sessions: 8931
Too short sessions (≤8): 5373
Invalid or missing labels: 0
Valid examples collected: 151527


In [19]:
str(action_vals)

"['session_end', 'application-window-opened', 'session_start', 'agency-dashboard::layout:render', 'agency-dashboard:::view', 'agency-dashboard::widget:render', 'agency-dashboard::configurable-table:render', '::nav-header:user-signed-out', 'dashboard:my-book:configurable-table:render', 'dashboard:my-book:widget:render', 'triaged-submission-list:my-book:configurable-table:render', 'triaged-submission-list:my-book::view', 'dashboard:my-book:layout:render', 'dashboard:my-book::view', '::nav-header:action-center-click', 'action-center:::view', 'account:::view', 'account-lines:::view', 'account-lines::layout:render', 'account-lines::widget:render', 'account-lines::configurable-table:render', ':all-accounts:configurable-table:render', ':all-accounts:widget:render', ':all-accounts:layout:render', ':all-accounts::view', 'submissions:policy-definition::submit-click', 'submissions:all-policy:configurable-table:render', 'submissions:all-policy::view', 'submissions:triaged_submissions-definition::v

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_all = torch.stack(X_examples) # (num_examples, time_steps, 34)
y_all = torch.tensor(y_examples, dtype=torch.long)

In [49]:
print(X_all.shape)
print(y_all.shape)

torch.Size([151527, 8, 34])
torch.Size([151527])


In [66]:
# Train/val/test split
num_examples = len(X_examples)
indices = torch.randperm(num_examples)

train_idx = indices[:int(0.8*num_examples)]
val_idx = indices[int(0.8*num_examples):int(0.9*num_examples)]
test_idx = indices[int(0.9*num_examples):]

X_train, y_train = X_all[train_idx], y_all[train_idx]
X_val, y_val = X_all[val_idx], y_all[val_idx]
X_test, y_test = X_all[test_idx], y_all[test_idx]

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

Training set: torch.Size([121221, 8, 34])
Validation set: torch.Size([15153, 8, 34])
Test set: torch.Size([15153, 8, 34])


In [94]:
class LSTM(nn.Module):
    def __init__(self, feature_sizes, embedding_dim=64, hidden_size=128):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=size, embedding_dim=embedding_dim)
            for size in feature_sizes
        ])
        self.rnn = nn.LSTM(
            input_size=embedding_dim * len(feature_sizes),
            hidden_size=hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, len(action_to_idx))
        
    def forward(self, x):
        # x shape: (batch_size, time_steps, num_features)
        batch_size, seq_len, num_feats = x.size()
        
        # Process each feature through embeddings
        embedded = []
        for i in range(num_feats):
            emb = self.embeddings[i](x[:, :, i])  # (batch_size, seq_len, emb_dim)
            embedded.append(emb)
        
        # Concatenate features
        x = torch.cat(embedded, dim=-1)  # (batch_size, seq_len, emb_dim*num_feats)
        
        # RNN processing
        out, _ = self.rnn(x)  # out: (batch_size, seq_len, hidden_size)
        
        # Use last timestep output for prediction
        logits = self.fc(out[:, -1, :])  # (batch_size, num_actions)
        return logits



In [95]:
# Initialize model
feature_sizes = [len(feature_to_idx[feat]) for feat in features]
model = LSTM(feature_sizes).to(device)

In [96]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params / 1e6:.2f}M")

Total parameters: 15.72M


In [97]:
# Create dataloaders
batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, y_val, )
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [98]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss/len(train_loader.dataset):.4f}")
    print(f"Val Loss: {val_loss/len(val_loader.dataset):.4f}")
    print(f"Val Acc: {100*correct/total:.2f}%")
    print("-------------------")

Epoch 1/10
Train Loss: 1.8766
Val Loss: 1.3514
Val Acc: 61.69%
-------------------
Epoch 2/10
Train Loss: 1.1904
Val Loss: 1.0949
Val Acc: 68.79%
-------------------
Epoch 3/10
Train Loss: 1.0182
Val Loss: 0.9713
Val Acc: 70.98%
-------------------
Epoch 4/10
Train Loss: 0.9294
Val Loss: 0.9057
Val Acc: 73.71%
-------------------
Epoch 5/10
Train Loss: 0.8662
Val Loss: 0.8859
Val Acc: 74.04%
-------------------
Epoch 6/10
Train Loss: 0.8124
Val Loss: 0.8679
Val Acc: 74.26%
-------------------
Epoch 7/10
Train Loss: 0.7596
Val Loss: 0.8416
Val Acc: 75.07%
-------------------


In [40]:
# Test evaluation
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(f"Test Loss: {test_loss/len(test_loader.dataset):.4f}")
print(f"Test Acc: {100*correct/total:.2f}%")

Test Loss: 2.2737
Test Acc: 32.90%
